# Análise Simples de Comentários

Objetivo: Descobrir quais amenities os usuários realmente mencionam e se falam bem ou mal delas.

In [ ]:
import pandas as pd
import json
from anthropic import Anthropic
import os
import re

In [24]:
# Setup Claude API
from dotenv import load_dotenv
load_dotenv()  # Carrega .env file

client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY")
)

# Verificar se API key foi carregada
if not os.environ.get("ANTHROPIC_API_KEY"):
    print("❌ API Key não encontrada!")
    print("Opções:")
    print("1. Crie arquivo .env com: ANTHROPIC_API_KEY=sua-chave")
    print("2. Ou defina: export ANTHROPIC_API_KEY='sua-chave'")
    print("3. Ou cole diretamente no código (menos seguro)")
else:
    print("✅ API Key carregada com sucesso")

✅ API Key carregada com sucesso


In [25]:
# Carregar dados - ESTRATÉGIA OTIMIZADA
reviews = pd.read_csv('data/processed/reviews.csv')
print(f"Total de reviews: {len(reviews)}")

# Amostra grande mas processável (respeitando limite de API)
sample_size = 5000  # Muito maior que antes
sample_reviews = reviews.sample(n=sample_size, random_state=42)
print(f"Amostra para análise: {len(sample_reviews)} comentários")
print(f"Estimativa de tempo: ~{sample_size/50:.1f} minutos (50 req/min limit)")

Total de reviews: 22376
Amostra para análise: 5000 comentários
Estimativa de tempo: ~100.0 minutos (50 req/min limit)


In [26]:
def clean_comment_for_api(comment):
    """Limpa e trunca comentário para eficiência máxima"""
    comment = re.sub(r'[\n\r\t]', ' ', str(comment))
    comment = comment.replace('"', "'")
    # Truncar para 100 caracteres (mais eficiente)
    return comment[:100]

def extract_amenities_batch(comments_batch):
    """Extrai amenities de MÚLTIPLOS comentários em uma única chamada API"""
    
    # Preparar prompt com múltiplos comentários
    comments_text = ""
    for i, comment in enumerate(comments_batch, 1):
        clean_comment = clean_comment_for_api(comment['comments'])
        comments_text += f"{i}. {clean_comment}\n"
    
    # Prompt ultra compacto para batch
    prompt = f"""Extract amenities from these comments:
{comments_text}

Return only valid JSON array:
[{{"comment":1,"amenity":"wifi","sentiment":"positive"}},{{"comment":1,"amenity":"kitchen","sentiment":"negative"}}]

Only extract physical amenities: wifi, kitchen, pool, AC, TV, washer, dryer, balcony, etc.
Ignore: location, host, apartment, building, neighborhood, price, service."""
    
    try:
        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=150,  # Aumentei um pouco para garantir resposta completa
            messages=[{"role": "user", "content": prompt}]
        )
        
        response_text = response.content[0].text.strip()
        
        # Tentar extrair JSON da resposta
        json_start = response_text.find('[')
        json_end = response_text.rfind(']') + 1
        
        if json_start != -1 and json_end > json_start:
            json_text = response_text[json_start:json_end]
            results = json.loads(json_text)
        else:
            print(f"Não foi possível encontrar JSON válido: {response_text[:100]}...")
            return []
        
        # Processar resultados
        processed_results = []
        for result in results:
            if isinstance(result, dict) and 'comment' in result and 'amenity' in result:
                comment_idx = result['comment'] - 1  # Converter para índice 0-based
                
                if 0 <= comment_idx < len(comments_batch):
                    original_comment = comments_batch[comment_idx]
                    
                    processed_results.append({
                        'item': result['amenity'],
                        'sentiment': result.get('sentiment', 'neutral'),
                        'listing_id': str(original_comment['listing_id']),
                        'review_id': str(original_comment['id'])
                    })
        
        return processed_results
        
    except json.JSONDecodeError as e:
        print(f"Erro JSON: {e}")
        print(f"Resposta recebida: {response_text[:200]}...")
        return []
    except Exception as e:
        print(f"Erro API: {e}")
        return []

# Teste com dados reais
print("Testando função corrigida...")
test_batch = sample_reviews.head(2).to_dict('records')
test_result = extract_amenities_batch(test_batch)
print(f"Resultado: {len(test_result)} amenities extraídas")
for r in test_result:
    print(f"  - {r['item']}: {r['sentiment']}")

Testando função corrigida...
Resultado: 1 amenities extraídas
  - kitchen: positive


In [27]:
# Processar comentários em LOTES - VERSÃO OTIMIZADA
import time
from datetime import datetime

all_amenities = []
errors_count = 0
processed_count = 0
batch_size = 10  # 10 comentários por requisição

# Converter para lista de dicionários para batch processing
comments_list = sample_reviews.to_dict('records')

print(f"Iniciando processamento em lotes de {batch_size} comentários...")
print(f"Total de lotes: {len(comments_list)//batch_size}")
print(f"Estimativa: ~{len(comments_list)//batch_size/50:.1f} minutos\n")

start_time = time.time()

# Processar em batches
for i in range(0, len(comments_list), batch_size):
    batch = comments_list[i:i+batch_size]
    batch_num = i//batch_size + 1
    
    print(f"Lote {batch_num}/{len(comments_list)//batch_size} ({len(batch)} comentários)...", end=' ')
    
    try:
        # Timing preciso: 1.2 segundos entre requisições (50 req/min)
        if batch_num > 1:  # Não esperar no primeiro lote
            time.sleep(1.2)
        
        batch_amenities = extract_amenities_batch(batch)
        all_amenities.extend(batch_amenities)
        processed_count += len(batch)
        
        print(f"✅ {len(batch_amenities)} amenities extraídas")
        
        # Checkpoint a cada 50 requisições (500 comentários)
        if batch_num % 50 == 0:
            checkpoint_file = f'data/processed/checkpoint_batch_{batch_num}.json'
            with open(checkpoint_file, 'w', encoding='utf-8') as f:
                json.dump(all_amenities, f, ensure_ascii=False, indent=2)
            
            elapsed = time.time() - start_time
            rate = processed_count / elapsed * 60  # comentários por minuto
            print(f"\n📊 Checkpoint {batch_num}: {len(all_amenities)} amenities | {rate:.0f} comentários/min")
    
    except Exception as e:
        errors_count += 1
        print(f"❌ Erro: {e}")
        continue

# Estatísticas finais
elapsed_time = time.time() - start_time
rate = processed_count / elapsed_time * 60

print(f"\n{'='*60}")
print(f"PROCESSAMENTO CONCLUÍDO!")
print(f"{'='*60}")
print(f"📊 Processados: {processed_count} comentários em {elapsed_time:.1f}s")
print(f"⚡ Taxa: {rate:.0f} comentários/minuto")
print(f"🎯 Amenities extraídas: {len(all_amenities)}")
print(f"❌ Erros: {errors_count}")
print(f"✅ Taxa de sucesso: {(1-errors_count/(len(comments_list)//batch_size))*100:.1f}%")

# Salvar resultados finais
try:
    with open('data/processed/amenities_from_comments.json', 'w', encoding='utf-8') as f:
        json.dump(all_amenities, f, ensure_ascii=False, indent=2)
    
    if all_amenities:
        df_amenities = pd.DataFrame(all_amenities)
        df_amenities.to_csv('data/processed/amenities_from_comments.csv', index=False, encoding='utf-8')
        print(f"💾 Dados salvos: amenities_from_comments.csv ({len(df_amenities)} linhas)")
    
except Exception as e:
    print(f"❌ Erro ao salvar: {e}")

Iniciando processamento em lotes de 10 comentários...
Total de lotes: 500
Estimativa: ~10.0 minutos

Lote 1/500 (10 comentários)... ✅ 2 amenities extraídas
Lote 2/500 (10 comentários)... Não foi possível encontrar JSON válido: [
  {
    "comment": 1,
    "amenity": "apartment",
    "sentiment": "positive"
  },
  {
    "commen...
✅ 0 amenities extraídas
Lote 3/500 (10 comentários)... ✅ 2 amenities extraídas
Lote 4/500 (10 comentários)... ✅ 1 amenities extraídas
Lote 5/500 (10 comentários)... Não foi possível encontrar JSON válido: [
  {
    "comment": 1,
    "amenity": "restaurants",
    "sentiment": "positive"
  },
  {
    "comm...
✅ 0 amenities extraídas
Lote 6/500 (10 comentários)... ✅ 5 amenities extraídas
Lote 7/500 (10 comentários)... ✅ 1 amenities extraídas
Lote 8/500 (10 comentários)... ✅ 4 amenities extraídas
Lote 9/500 (10 comentários)... ✅ 1 amenities extraídas
Lote 10/500 (10 comentários)... Não foi possível encontrar JSON válido: [
  {
    "comment": 1,
    "amenity": "kitc